In [ ]:
import   os
import   sys
import   numpy          as       np
import   ismrmrd        as       mrd
import   ismrmrd.xsd
from     matplotlib     import   pyplot   as plt
import   h5py
import   nibabel        as       nib

## First - read in file with reconstructed image data

In [ ]:
dataset_to_convert = 'reconstructed_data.h5'

In [ ]:
f = h5py.File(dataset_to_convert, 'r')

In [ ]:
f

In [ ]:
f.keys()

In [ ]:
hdf_groups_keys_in_file = list(f.keys())

In [ ]:
hdf_groups_keys_in_file

In [ ]:
for key in hdf_groups_keys_in_file:
   print(f[key])

In [ ]:
group_sub_keys = list()

for group_key in hdf_groups_keys_in_file:
   group_sub_keys.append(list(f[group_key].keys()))

In [ ]:
group_sub_keys

In [ ]:
len(group_sub_keys)

In [ ]:
test_keys = list(f['2021-12-21 16:24:14']['image_0'].keys())

In [ ]:
test_keys

In [ ]:
f['2021-12-21 16:24:14']['image_0']['data']

In [ ]:
f['2021-12-21 16:24:14']['image_0']['header']

In [ ]:
datasets = []

for group_key in hdf_groups_keys_in_file:
   # dataset_views = []
   # print("group key is: %s" % str(group_key))
   for i in range(len(group_sub_keys)):

      for key in group_sub_keys[i]:
         # dataset_views.append(f[group_key][key])
         # print("data set: %s" % f[group_key][key])
         view_keys = list(f[group_key][key].keys())

         for member in range(len(view_keys)):
            # print("key %d: %s" % (member, view_keys[member]))
            member_key = f[group_key][key][view_keys[member]]

            datasets.append(member_key)

datasets

In [ ]:
datasets[2][0]

In [ ]:
datasets[2][0]['matrix_size']

In [ ]:
f.close()

### Following [this](https://gadgetron.discourse.group/t/reading-ismrmrd-image-type/322/4) suggestion from David Hansen

In [ ]:
f = ismrmrd.File('reconstructed_data.h5')

In [ ]:
# First, we need to find all image groups / sets within the ISMRMRD image HDF5 file

image_sets = [f[image_set].images for image_set in f.find_images()]

In [ ]:
len(image_sets)

In [ ]:
# This should now tell us how many images are present within a give image group/set.
# Should look up a better way to do this iteration.

image_sets[0].headers.size

In [ ]:
header_keys_for_nifti = [
  "matrix_size",
  "slice",
  "read_dir",
  "phase_dir",
  "slice_dir",
  "position",
  "repetition",
  "contrast",
  "data_type",
  "image_type",
  "channels",
  "flags"
]

# Iterate over all sets within the ISMRMRD file. This should generate 1 NIFTI file per group (to be later confirmed)
for image_set in image_sets:
   # Now iterate over all images within each set.  Hopefully each image within a set has the same dimensions
   for j in range(image_set.headers.size):
      for header_value_key in header_keys_for_nifti:
         print ("Header value %16s from image set %d is: %s" % (header_value_key, j, image_set.headers[j][header_value_key]))

### Now create NIFTI-1 dataset from one of the data arrays read in from the ISMRMRD image sets.

In [ ]:
# Transpose seems to be necessary to pack data properly

nii_image_data = image_sets[0].data[0::].transpose(4,3,2,1,0)

In [ ]:
# Now, create data set in memory from image data.  Should get basic data dimensions and data types.  This should
# leave the geometry and position information to be computed and filled in.

new_nii = nib.Nifti1Image(nii_image_data, None)

In [ ]:
print(new_nii.header)

In [ ]:
# Can access NIFTI header elements through keys, similar to how the ISMRMRD image header values above are accessed.

new_nii.header['dim']

In [ ]:
# Can change these, from ISMRMRD header values, as needed.

new_nii.header['dim'] = [  5, 256, 256,   3,   1,   1,   1,   1]

In [ ]:
nib.save(new_nii, 'new_test_nifti.nii')